### Sources
**Unless otherwise stated in the cells, the code in this notebook is written by me (Sonja Brehm).**
Other sources are marked with **##** at the beginning of the corresponding code cell.
____

# Clean up the dataset
## Joining split manifestos back together
Currently, some of the manifestos (the newer ones) are split into many quasi-sentences/rows because the Manifesto Project has annotated them. This is why my dataset partly looks like this (simplified):

    manifesto_id                                      text  date
    101                             The government changed  1990
    101                 the legislation on import control.  1990
    102                 Politics cannot solve all problems  2005
    102                  but it should try to do its part.  2005
    
The following code puts all the "text" fields of a manifesto back together.

In [2]:
import pandas as pd 
import numpy as np

In [3]:
#Read the CSV file into a pandas dataframe
df = pd.read_csv('data/manifestos.csv', encoding='utf-8')

#show 3 sample rows
df.sample(3)

,Unnamed: 0,text,cmp_code,eu_code,pos,manifesto_id,party,date,language,source,...,is_primary_doc,may_contradict_core_dataset,md5sum_text,url_original,md5sum_original,annotations,handbook,is_copy_of,title,id
58162,58163,„Gesundheit – Made in Germany“ könnte ein neue...,601,NaN,669,41420_200509,41420,200509,german,MARPOR,...,True,False,b2cc65f8510721c2c5287efc311898a7,/down/originals/41420_2005.pdf,ccc9b7c3e02e1815497395e4accf0539,True,2.0,NaN,Arbeit hat Vorfahrt. Deutschlandprogramm 2005,41420_200509
5998,5999,Menschenrechte ohne Rabatt,NaN,NaN,1867,41113_200509,41113,200509,german,MARPOR,...,True,False,f3e28bd1c8921bda6ff7c34e0f7b6e93,/down/originals/41113_2005.pdf,5095c390123feccb8526b7619b61daa6,True,2.0,NaN,Eines für alle: Das Grüne Wahlprogramm 2005,41113_200509
36289,36290,und dem Bedarf für inklusive Bildung anpassen.,503,NaN,1541,41223_202109,41223,202109,german,MARPOR,...,True,False,fe0259d4c039a20854710fa77f0bc051,/down/originals/2021-1/41223_2021.pdf,33b510bc5817b18e648e0c8ea20481de,True,5.0,NaN,"Zeit zu handeln! Für soziale Sicherheit, Fried...",41223_202109


In [4]:
np.shape(df) #88750 rows, 21 columns

(88750, 21)

In [5]:
#only keep the columns we are interested in for the project
df = df[['text', 'manifesto_id', 'party', 'date', 'title']]

In [6]:
## Code in this cell is from stackoverflow, username of the answer: EdChum, question: 'Concatenate strings from several rows using Pandas groupby'
## link: https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
## modified to use my data and join fragments with a space instead of a comma

#merge the "text" column of all rows that have the same manifesto_id, date, party & title
#and add a space when joining the text fragments together
df_new = df.groupby(['manifesto_id','date', 'party', 'title'])['text'].apply(lambda x: ' '.join(x)).reset_index()
df_new

,manifesto_id,date,party,title,text
0,41111_198303,198303,41111,Diesmal Die Grünen - Warum? Ein Aufruf zur Bun...,DIESMAL DIE GRÜNEN. WARUM? Ein Aufruf zur Bun...
1,41111_198701,198701,41111,Farbe bekennen. Bundestagswahlprogramm 1987,Bundestagswahl Programm 1987 Farbe bekennen ...
2,41112_199012,199012,41112,1. Die Grünen. Das Programm zur 1. gesamtdeuts...,Das Programm zur 1. Gesamtdeutschen Wahl 1990....
3,41113_199410,199410,41113,Bundestagswahlprogramm,BÜNDNIS 90/DIE GRÜNEN PROGRAMM ZUR BUNDESTAGS...
4,41113_199809,199809,41113,Programm zur Bundestagswahl 1998. Grün ist der...,BÜNDNIS 90/DIE GRÜNEN Grün ist der Wechsel Pro...
...,...,...,...,...,...
91,41951_195309,195309,41951,Programm des Gesamtdeutschen Blocks/BHE für di...,Programm des Gesamtdeutschen Blocks/BHE für di...
92,41952_201309,201309,41952,Wir stellen das mal infrage. Wahlprogramm zur ...,Einleitung Die Bundestagswahl am 22. September...
93,41953_201309,201309,41953,Wahlprogramm,• Wir fordern eine geordnete Auflösung des Eu...
94,41953_201709,201709,41953,Programm für die Wahl zum Deutschen Bundestag ...,KAPITEL 1 Verteidigung der Demokratie in Deuts...


In [7]:
#check the text of a manifesto from 2021 (that had annotations) –> looks good, sentence structure makes sense
df_new.loc[df_new["date"]==202109] #show all manifestos from 2021
df_new.at[10,'text'][1000:4000] #...and show a part of the text of one of them

'ichtung sie nimmt. Wir, BÜNDNIS 90/DIE GRÜNEN, legen mit diesem Programm unser inhaltliches Angebot an Sie vor. Wir tun dies in einer Zeit des globalen Ausnahmezustands. Die Pandemie hat uns alle bis ins Mark getroffen. Sie hat im Guten gezeigt, zu welcher Gemeinsamkeit, Innovationskraft und Widerstandsfähigkeit wir Menschen erreichen können. Sie hat aber auch die Schwachstellen unserer Gesellschaft schonungslos offengelegt, und das in einer ohnehin verwundbaren Welt. Die globalen Krisen dieser Zeit – zuallererst die Klimakrise als wahre Menschheitskrise – wirken in unser aller Leben hinein und gefährden Freiheit, Sicherheit und Wohlstand. Wir haben aber die Wahl: Wir können entscheiden, ob uns die Krisen über den Kopf wachsen oder wir über sie hinaus. Die Erfahrungen in der Pandemie zeigen, dass wir Krisen in gemeinsamer Kraftanstrengung bewältigen können. Durch die Solidarität, mit der unsere Gesellschaft den Verletzlichsten den stärksten Schutz gegeben hat. Durch Rücksichtnahme, di

## Sort out manifestos
For their dataset, the Manifesto Project chose the election programs of parties that gained at least one seat in parliament at that election. I want to reduce this selection for my project by only including parties that gained at least one seat in three elections. Therefore, I have to delete all other parties that do not meet this requirement from the dataset.

Some parties have had name changes and mergers, meaning the same party can have different party IDs. Therefore, I had to go through the dataset by hand and found the following parties that I needed to delete: 
- 41220 (Communist Party of Germany)
- 41522 (Center Party)
- 41711 (Economic Reconstruction League)
- 41712 (German Reich Party)
- 41911 (Bavarian Party)
- 41912 (South Schleswig Voters’ Union)
- 41951 (Refugee Party)
- 41952 (Pirates)

Two parties were elected twice, so my dataset should end up with 10 rows less.

In [8]:
#create a list of all party IDs to be deleted
delete_parties = [41220, 41522, 41711, 41712, 41911, 41912, 41951, 41952]

#loop trough all indices of the dataset
for i in range(len(df_new)):
    #get the party ID of this index/row
    party = df_new.at[i, 'party']
    #if the party ID is in delete_parties, delete the row
    if party in delete_parties:
        df_new.drop(i, inplace=True)
#reset the indices so that they range from 0 to len(df_new) again
df_new.reset_index(drop=True, inplace=True)

In [9]:
#Check if it worked - you can see that e.g. the former row 92 with party ID "41952" got deleted
#and we have 10 rows less than before
df_new

,manifesto_id,date,party,title,text
0,41111_198303,198303,41111,Diesmal Die Grünen - Warum? Ein Aufruf zur Bun...,DIESMAL DIE GRÜNEN. WARUM? Ein Aufruf zur Bun...
1,41111_198701,198701,41111,Farbe bekennen. Bundestagswahlprogramm 1987,Bundestagswahl Programm 1987 Farbe bekennen ...
2,41112_199012,199012,41112,1. Die Grünen. Das Programm zur 1. gesamtdeuts...,Das Programm zur 1. Gesamtdeutschen Wahl 1990....
3,41113_199410,199410,41113,Bundestagswahlprogramm,BÜNDNIS 90/DIE GRÜNEN PROGRAMM ZUR BUNDESTAGS...
4,41113_199809,199809,41113,Programm zur Bundestagswahl 1998. Grün ist der...,BÜNDNIS 90/DIE GRÜNEN Grün ist der Wechsel Pro...
...,...,...,...,...,...
81,41620_195309,195309,41620,Wohl und Ehre des deutschen Volkes und Vaterla...,Arbeitsprogramm für den zweiten Deutschen Bund...
82,41620_195709,195709,41620,Wahlprogramm der Deutschen Partei (DP/FVP),WAHLPROGRAMM der Deutschen Partei (DP/FVP) (ve...
83,41953_201309,201309,41953,Wahlprogramm,• Wir fordern eine geordnete Auflösung des Eu...
84,41953_201709,201709,41953,Programm für die Wahl zum Deutschen Bundestag ...,KAPITEL 1 Verteidigung der Demokratie in Deuts...


In [10]:
#export as new CSV
df_new.to_csv('data/sorted_manifestos.csv')